# Arxiv Chatbot

## Overview

This project implements a chatbot system with integrated tool definitions and execution. The chatbot is designed as a simple but extensible example for understanding how conversational AI can be augmented with tools, via model context protocol (MCP) to perform tasks beyond basic dialogue.

## Import libraries

In [3]:
import arxiv
import json
import os
from typing import List
from dotenv import load_dotenv
import anthropic